In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
from nltk.corpus import stopwords

Function to lower case and remove punctuations.

In [2]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Tokenization
    words = text.split()
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    return ' '.join(words)

In [3]:
#reading the files
customer_db = pd.read_excel('your path.xlsx')  # customer database file
transactions = pd.read_excel('your path.xlsx') # transactions file

In [4]:
# Preprocessing text and converting to list
customer_db['names_prepped'] = customer_db["Customer Names"].astype(str).apply(preprocess_text)
customer_names = customer_db['names_prepped'].tolist()
transactions['details_prepped'] = transactions['Remitter details'].astype(str).apply(preprocess_text)
remitter_details = transactions['details_prepped'].tolist()

In [5]:
# tfidf_customer_names and tfidf_remitter_details contain the respective vectors
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3)).fit(customer_names + remitter_details)
tfidf_customer_names = vectorizer.transform(customer_names)
tfidf_remitter_details = vectorizer.transform(remitter_details)

In [6]:
#Calculating cosine similarity between the two vectors
cosine_similarities = cosine_similarity(tfidf_remitter_details, tfidf_customer_names)

In [7]:
results = []

In [8]:
# Iterate over each name in the other file
for index, row in transactions.iterrows():
    preprocessed_name = preprocess_text(str(row['Remitter details']))
    
        # Find the best match in customer database
    max_similarity_index = cosine_similarities[index].argmax()
    index_list = np.argwhere(cosine_similarities[index] == cosine_similarities[(index,max_similarity_index)]).flatten().tolist()
        
        

    if len(index_list) == 1:
        
        best_match_name = customer_names[max_similarity_index]
        best_match_info = customer_db.iloc[max_similarity_index]
        results.append({
                    'Transaction date': row['Transaction date'],
                    'Reference number': row['Reference Number'],
                    'Remitter details': row['Remitter details'],
                    'Amount' : row['Amount (INR)'],
                    'Customer name': customer_db['Customer Names'][max_similarity_index],
                    'Customer Code': customer_db['Customer Number'][max_similarity_index],
                    'Similarity': cosine_similarities[(index,max_similarity_index)],
                    'Count' : 1  #count is included to see how many customers are there with the same percentage match for a specific remitter name
                })
                          
    else:

        if cosine_similarities[(index,max_similarity_index)]!=0 :
                
            index_list_length = len(index_list)
            for i in range(index_list_length):
                    
                    results.append({
                    'Transaction date': row['Transaction date'],
                    'Reference number': row['Reference Number'],
                    'Remitter details': row['Remitter details'],
                    'Amount' : row['Amount (INR)'],
                    'Customer name': customer_db['Customer Names'][max_similarity_index],
                    'Customer Code': customer_db['Customer Number'][max_similarity_index],
                    'Similarity': cosine_similarities[(index,max_similarity_index)],
                    'Count' : i+1
                        }
                                
                              )
        else:
            continue

# Create a DataFrame with the results
results_df = pd.DataFrame(results)
print(results_df)

   Transaction date Reference number           Remitter details   Amount  \
0        2024-02-02           123ABC          DR PATTI HOSPITAL     5000   
1        2024-02-03           234DEF  National Institute of Men   100000   
2        2024-02-04           456EFD          SANTOSHKUMARSINGH   501108   
3        2024-02-05           678ABC              AARTI A SINGH    40000   
4        2024-02-06           666BVG                   AMBANI S   456732   
5        2024-02-07           789ION              Miss B SHUKLA    78000   
6        2024-02-08           665YUB        FIT n Fine Bangalor     2398   
7        2024-02-09           778NMJ       REJUVANATE MULTISPEC  1200000   
8        2024-02-10           345TRE  THEHOPE HOSPITAL AND HEAL   100500   
9        2024-02-11           234DEF        BEST CLINIC PVT LTD   100020   
10       2024-02-12           234DEH             CleanGreenHosp      678   

                                       Customer name  Customer Code  \
0               

In [9]:

# determining the name of the file
file_name = 'TextMatching.xlsx'
 
# saving the excel
results_df.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
